In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Dataset/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')


print(train_df.head())
print(test_df.head())

          ID     label    0    1    2    3    4    5    6    7  ...  1014  \
0  TRAIN_000  building  251  196   51   63   68   78  189   34  ...    85   
1  TRAIN_001  building  247  184  203  237  255  255  193  255  ...   242   
2  TRAIN_002  building  217  223  232  231  239  212  109  115  ...    96   
3  TRAIN_003       cat  133  149  153  138   68  157  159  166  ...   245   
4  TRAIN_004  building  240  213  187  159  112  134  239  233  ...   148   

   1015  1016  1017  1018  1019  1020  1021  1022  1023  
0   195    63    30    73    65    63   201   251   248  
1   239   241   242   242   241   241   241   240   238  
2    90   103   166   191   163   190   190   206   231  
3   241   247   255   250   190   186   244   254   201  
4    59   163   133    92   196   221   194   182   251  

[5 rows x 1026 columns]
         ID    0    1    2    3    4    5    6    7    8  ...  1014  1015  \
0  TEST_000  186  189  189  190  190  190  192  191  192  ...   200   200   
1  TEST_00

In [ ]:
label_encoder = LabelEncoder()


train_df['label'] = label_encoder.fit_transform(train_df['label'])


print(train_df['label'].head())


print(f"Data type of label: {type(train_df['label'].iloc[0])}")

0    4
1    4
2    4
3    5
4    4
Name: label, dtype: int64
Data type of label: <class 'numpy.int64'>


In [ ]:
class IconDataset(Dataset):
    def __init__(self, dataframe, transform=None, has_label=True):
        self.data = dataframe
        self.transform = transform
        self.has_label = has_label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        img = self.data.iloc[idx, 1:].values.astype(np.uint8)
        img = img[:1024]
        img = img.reshape(32, 32)


        if self.has_label:
            label = self.data.iloc[idx]['label']
        else:
            label = None


        if self.transform:
            img = self.transform(img)


        img = torch.tensor(img, dtype=torch.float32)

        if self.has_label:
            label = torch.tensor(label, dtype=torch.long)
            return img, label
        else:
            return img

In [ ]:
train_data, val_data = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['label']
)


train_dataset = IconDataset(train_data)
val_dataset = IconDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


test_dataset = IconDataset(test_df, has_label=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)


        self.downsample = None
        if in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))


        if self.downsample is not None:
            residual = self.downsample(residual)
        out += residual
        out = torch.relu(out)
        return out

class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_pool):
        super(InceptionBlock, self).__init__()
        self.branch1 = nn.Conv2d(in_channels, out_1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, red_3x3, kernel_size=1),
            nn.Conv2d(red_3x3, out_3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, red_5x5, kernel_size=1),
            nn.Conv2d(red_5x5, out_5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool, kernel_size=1)
        )

    def forward(self, x):
        out1 = torch.relu(self.branch1(x))
        out2 = torch.relu(self.branch2(x))
        out3 = torch.relu(self.branch3(x))
        out4 = torch.relu(self.branch4(x))

        out = torch.cat([out1, out2, out3, out4], dim=1)
        return out

class IconCNN(nn.Module):
    def __init__(self):
        super(IconCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.block1 = ResidualBlock(32, 32)
        self.inception_block = InceptionBlock(32, 32, 32, 64, 16, 32, 32)
        self.block2 = ResidualBlock(160, 64)
        self.block3 = ResidualBlock(64, 128)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 10)

        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.block1(x)
        x = self.inception_block(x)
        x = self.pool(x)
        x = self.block2(x)
        x = self.pool(x)
        x = self.block3(x)
        x = self.pool(x)

        x = x.view(-1, 128 * 2 * 2)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
model = IconCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


IconCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): ResidualBlock(
    (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (inception_block): InceptionBlock(
    (branch1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (branch2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (branch3): Sequential(
      (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (branch

In [ ]:
early_stopping_counter = 0
best_val_accuracy = 0.0

In [ ]:
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=40):
    global early_stopping_counter, best_val_accuracy
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for data in tqdm(train_loader):
            images, labels = data
            images = images.unsqueeze(1).float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")


        val_accuracy = evaluate_model(model, val_loader)


        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= 5:
            print("Early stopping triggered. Training will stop.")
            break


        scheduler.step(running_loss)


def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data[0], data[1]
            images, labels = images.unsqueeze(1).float().to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")
    return val_accuracy


train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=30)


evaluate_model(model, val_loader)

100%|██████████| 39/39 [00:01<00:00, 21.35it/s]


Epoch [1/30], Loss: 1.6872, Accuracy: 42.28%
Validation Accuracy: 55.84%


100%|██████████| 39/39 [00:01<00:00, 21.04it/s]


Epoch [2/30], Loss: 0.7497, Accuracy: 74.47%
Validation Accuracy: 61.04%


100%|██████████| 39/39 [00:01<00:00, 21.45it/s]


...

100%|██████████| 39/39 [00:01<00:00, 21.43it/s]


Epoch [14/30], Loss: 0.0008, Accuracy: 100.00%
Validation Accuracy: 92.21%


100%|██████████| 39/39 [00:02<00:00, 18.99it/s]


Epoch [15/30], Loss: 0.0005, Accuracy: 100.00%
Validation Accuracy: 92.86%
Early stopping triggered. Training will stop.
Validation Accuracy: 92.86%


92.85714285714286

In [ ]:
def generate_submission(model, test_loader, test_df, output_file='submission.csv'):
    model.eval()
    predictions = []

    with torch.no_grad():
        for data in tqdm(test_loader):
            images = data.unsqueeze(1).float().to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())


    predicted_labels = label_encoder.inverse_transform(predictions)


    submission_df = pd.DataFrame({
        'ID': test_df['ID'],
        'label': predicted_labels
    })


    submission_df.to_csv(output_file, index=False)
    print(f'Submission file saved to {output_file}')


test_df = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')
generate_submission(model, test_loader, test_df, output_file='/content/drive/MyDrive/Dataset/submission.csv')


100%|██████████| 16/16 [00:00<00:00, 44.92it/s]

Submission file saved to /content/drive/MyDrive/Dataset/submission.csv
